In [1]:
import sys
import os
from pathlib import Path

# add the project to python path so that we can import src
cwd = Path().resolve()

if cwd.name == "notebooks":
    project_root = cwd.parent
else:
    project_root = cwd

if str(project_root) not in sys.path:
    sys.path.insert(0,str(project_root))

#insert(0, ...) places the new path at the beginning of the list, making it the first directory Python checks when an import statement is executed

In [2]:
sys.path

['/Users/amruthakaruturi/gitrepos/Full-Stack-RAG-project/server',
 '/opt/homebrew/Cellar/python@3.11/3.11.14_1/Frameworks/Python.framework/Versions/3.11/lib/python311.zip',
 '/opt/homebrew/Cellar/python@3.11/3.11.14_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11',
 '/opt/homebrew/Cellar/python@3.11/3.11.14_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload',
 '',
 '/Users/amruthakaruturi/Library/Caches/pypoetry/virtualenvs/six-figure-rag-api-5KEfUhx6-py3.11/lib/python3.11/site-packages']

In [3]:
from langchain.agents import create_agent
from langchain.tools import tool
from src.rag.retrieval.index import retrieve_context
from src.rag.retrieval.utils import prepare_prompt_and_invoke_llm
from langgraph.graph import MessagesState
from typing import Any, List, Dict
from typing_extensions import Annotated
from langgraph.types import Command
from langchain_core.tools.base import InjectedToolCallId
from langchain_core.messages import ToolMessage 


/Users/amruthakaruturi/Library/Caches/pypoetry/virtualenvs/six-figure-rag-api-5KEfUhx6-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Create a custom agent state that extends the MessagesState to store citations
class CustomAgentState(MessagesState):
	"""Extended agent state with citations tracking"""
	# citations will accumulate across tool calls
	citations: Annotated[List[Dict[str, Any]], lambda x, y: x + y] = []

In [16]:
# InjectedToolCallIdThis annotation is used to mark a tool parameter that should receive the tool call ID at runtime.

def create_rag_tool(project_id:str):
    """
    Create a RAG search tool bound to a specific project.
    
    This factory function creates a tool that is bound to a specific project_id,
    allowing the agent to search through that project's documents.
    
    Args:
        project_id: The UUID of the project whose documents should be searchable
        
    Returns:
        A LangChain tool configured for RAG search on the specified project
        
    Example:
        >>> rag_tool = create_rag_tool("123e4567-e89b-12d3-a456-426614174000")
    """
    @tool
    def rag_search(
        query: str,
        tool_call_id: Annotated[str, InjectedToolCallId],
    ) -> Command:
        """
        Search through project documents using RAG (Retrieval-Augmented Generation).
        This tool retrieves relevant context from the current project's documents based on the query.
        
        Args:
            query: The search query or question to find relevant information
            tool_call_id: Injected tool call ID for message tracking
            
        Returns:
            A Command object with updated messages and citations
        """
        try: 
            texts, images, tables, citations = retrieve_context(project_id, query)
            if not texts:
                return Command[tuple[()]](
                    update={
                        "messages": [
                            ToolMessage(
                                "No relevant informatin found in the project documents for this query",
                                tool_call_id = tool_call_id
                            )
                        ]
                    }
                )
            response = prepare_prompt_and_invoke_llm(
                user_query = query,
                texts = texts,
                images = images,
                tables = tables
            )
            return Command(
                update={
                    # update message history
                    "messages":[
                        ToolMessage(
                            content = response,
                            tool_call_id = tool_call_id
                        )
                    ],
                    # update citations in state - these accumulate
                    "citations": citations
                }
            )
                
        except Exception as e:
            return Command(
                update={
                    "messages": [
                        ToolMessage(
                            f"Error retrieving information: {str(e)}",
                            tool_call_id=tool_call_id
                        )
                    ]
                }
            )
        
    return rag_search


In [6]:
# Create agent with project-specific RAG tool
def create_rag_agent(project_id: str, model: str = "gpt-4o"):
	"""Create an agent with RAG tool for a specific project"""
	
	# Create tools list with project-specific RAG tool
	tools = [create_rag_tool(project_id)]
	
	# Define the system prompt
	system_prompt = """You are a helpful AI assistant with access to a RAG (Retrieval-Augmented Generation) tool that searches project-specific documents.

For every user question:

1. Do not assume any question is purely conceptual or general.  
2. Use the `rag_search` tool immediately with a clear and relevant query derived from the user’s question.  
3. Carefully review the retrieved documents and base your entire answer on the RAG results.  
4. If the retrieved information fully answers the user’s question, respond clearly and completely using that information.  
5. If the retrieved information is insufficient or incomplete, explicitly state that and provide helpful suggestions or guidance based on what you found.  
6. Always present answers in a clear, well-structured, and conversational manner.

**Never answer without first querying the RAG tool. This ensures every response is grounded in project-specific context and documentation.**
"""
	
	# Create the agent graph
	agent = create_agent(
		model=model,
		tools=tools,
		system_prompt=system_prompt,
		state_schema=CustomAgentState
	)
	
	return agent

In [7]:
project_id = "a040a0e5-35eb-48ec-bef1-8c567c98b3a6"
rag_agent = create_rag_agent(project_id=project_id, model="gpt-4o")

In [8]:
inputs = {"messages":[{"role":"user","content": "What are the two types of sleep?"}]}
result = rag_agent.invoke(inputs)

Vector search resulted in: 3 chunks
🤖 Invoking LLM with 2 messages (3 texts, 0 tables, 0 images)...


In [9]:
result["messages"][-1]

AIMessage(content='There are two main types of sleep: non-REM (NREM) sleep and REM (rapid eye movement) sleep.\n\n1. **Non-REM (NREM) Sleep:** This type consists of three stages that range from light to deep sleep. The deepest stage is Stage 3 NREM, also known as slow-wave sleep.\n\n2. **REM Sleep:** This stage is characterized by rapid eye movements, vivid dreams, temporary muscle paralysis, and brain activity that resembles wakefulness. \n\nThese two types cycle throughout the sleep period, contributing to a restful and restorative sleep experience.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 391, 'total_tokens': 509, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deac

In [12]:
result

{'messages': [HumanMessage(content='What are the two types of sleep?', additional_kwargs={}, response_metadata={}, id='9d6cdf0e-a1e8-42ba-8140-bd40534304f8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 286, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_deacdd5f6f', 'id': 'chatcmpl-Cza1Xygmo5Oxhcvmn2SpQIsokRENv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bd43f-ee2b-70c0-ba0f-03dbd9061a1c-0', tool_calls=[{'name': 'rag_search', 'args': {'query': 'types of sleep'}, 'id': 'call_KqKN7X7GqyhPW5j8w3XO8nQQ', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 28